In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from fbprophet import Prophet
from sklearn.cross_validation import KFold
from service.files_service import _get_files

/home/ramon/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
PATH = 'data/brent_wti/resampled_D/'

In [3]:
all_files = _get_files(PATH ,extension='.csv',filter_on='_H_')

In [4]:
def check_date_range(file_name):
    df = pd.read_csv(PATH+'resampled_D/'+file_name,parse_dates=['ds'])
    print df.ds.min(),df.ds.max()

In [5]:
def resampler(file_name):
    df = pd.read_csv(PATH+file_name)
    df['ds'] = pd.to_datetime(df['Time (UTC)'],format='%Y.%m.%d %H:%M:%S')
    df = df.set_index('ds').resample('H')
    df.to_csv(PATH+'/resampled_H_'+file_name)

In [15]:
df = pd.read_csv(PATH+all_files[0])
#df['ds'] = pd.to_datetime(df['Time (UTC)'],format='%Y.%m.%d %H:%M:%S')
test_range = 100

In [16]:
df_train = df[:-test_range]
df_test = df[-test_range:]
print df_train.ds.min(),"--",df_train.ds.max()
print df_test.ds.min(),"--",df_test.ds.max()


2011-09-23 07:00:00 -- 2016-12-04 19:00:00
2016-12-04 20:00:00 -- 2016-12-08 23:00:00


In [17]:
df_train['y'] = np.log(np.exp(df_train.Close))

/home/ramon/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
m = Prophet(changepoints=None,
            n_changepoints=25,
            yearly_seasonality=True,
            weekly_seasonality=True,
            holidays=None,
            seasonality_prior_scale=10.0,
            holidays_prior_scale=10.0,
            changepoint_prior_scale=0.05,
            mcmc_samples=0,
            interval_width=0.80,
            uncertainty_samples=1000,)
m.fit(df_train)

In [ ]:
# Python
future = m.make_future_dataframe(periods=test_range, freq='H')

In [ ]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast.tail(test_range).set_index('ds')['yhat'].plot(color='g')
df_test.set_index('ds')['Close'].plot(color='r')

In [ ]:
# Python
m.plot(forecast)